In [ ]:
%attachments

In [9]:
%%spTraining
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
import os

# Prepare training documents from a list of (id, text, label) tuples.
def getProjectRepo():
    repo = os.popen('bdvcli --get cluster.project_repo').read().rstrip()
    if not repo:
        print("unable to find, project repo, some history will be lost")
        return 'file:///tmp/'
    return 'file://' + repo

training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)
#print(type(model))
model.save(getProjectRepo() + '/' + 'models/spark_models/TextPredction.model1')

# Prepare test documents, which are unlabeled (id, text) tuples.
# test = spark.createDataFrame([
#     (4, "spark i j k"),
#     (5, "l m n"),
#     (6, "spark hadoop spark"),
#     (7, "apache hadoop")
# ], ["id", "text"])

In [15]:
%%sciKitTraining

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os

def getProjectRepo(path):
    ProjectRepo = os.popen('bdvcli --get cluster.project_repo').read().rstrip()
    return ProjectRepo + '/' + path        
# fix random seed for reproducibility

numpy.random.seed(7)
# load pima indians dataset
dataset = numpy.loadtxt(getProjectRepo('data/pima-indians-diabetes.csv'), delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X, Y, epochs=150, batch_size=10, verbose=0)
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model_json = model.to_json()
with open(getProjectRepo('models/demo1/model.json'), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(getProjectRepo('models/demo1/model.h5'))
print("Saved model to disk")


Fetching results from training cluster:sciKitTraining
Received results from training cluster:sciKitTraining
{"id":4,"input":"['', 'from keras.models import Sequential', 'from keras.layers import Dense', 'from keras.models import model_from_json', 'import numpy', 'import os', '', 'def getProjectRepo(path):', \"    ProjectRepo = os.popen('bdvcli --get cluster.project_repo').read().rstrip()\", \"    return ProjectRepo + '/' + path        \", '# fix random seed for reproducibility', '', 'numpy.random.seed(7)', '# load pima indians dataset', 'dataset = numpy.loadtxt(getProjectRepo(\\'data/pima-indians-diabetes.csv\\'), delimiter=\",\")', '# split into input (X) and output (Y) variables', 'X = dataset[:,0:8]', 'Y = dataset[:,8]', '# create model', 'model = Sequential()', \"model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))\", \"model.add(Dense(8, kernel_initializer='uniform', activation='relu'))\", \"model.add(Dense(1, kernel_initializer='uniform', activation=